In [ ]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
file_path = 'roblox_games_data.csv'
# Load the latest version
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "databitio/roblox-games-data",
    file_path
)

In [ ]:
# basic data cleaning (remove duplicate and null data in required row)
start_len = len(df)

df = df.drop_duplicates() #remove duplicate rows

new_len = len(df) - start_len
print(f'{new_len} rows duplicate')

df = df.dropna(subset=['Title','gameID','Active Users','Favorites','Genre']) # remove rows which missing column

new_len = len(df) - start_len - new_len
print(f'{new_len} rows missing required column data')

# reset index
df = df.reset_index(drop=True)

print(f'{len(df) - start_len} total rows removed')
print(f'{len(df)} total rows remaining')

In [ ]:
# List all unique genres
if "Genre" in df.columns:
    genres = df["Genre"].dropna().unique()
    print("All genres:", genres)
else:
    print("No 'Genre' column found.")

In [ ]:
# print debug to understand data set
print(df.columns.to_list())
print(df['Title'].value_counts())
print(df['Total Visits'].value_counts())

In [ ]:
# analyze duplicate data of one game
def analyze_game_data(selected_game,df):
    game = df[df['Title'] == selected_game]

    if len(game) == 0:
        print(f'{selected_game} not found')
        return
    
    print(f'Total row(s) (duplicated) of game {game['Title'].value_counts()}')
    print('----------------------------------')
    print(f'Data of {game['Date Created'].value_counts()}') 
    print('----------------------------------')
    print(f'Data of {game['Last Updated'].value_counts()}') 
    print('----------------------------------')
    print(f'Data of {game['Favorites'].value_counts()}') 
    print('----------------------------------')
    print(f'Data of {game['Active Users'].value_counts()}') 
    print('----------------------------------')
    print(f'Data of {game['Description'].value_counts()}') 
    print('----------------------------------')
    print(f'Data of {game['Date'].value_counts()}') 
    print('----------------------------------')

In [ ]:
def parse_visits(val):
    # Handle B+, M+, K+ etc.
    if isinstance(val, str):
        if 'B' in val:
            return float(val.replace('B+', '').replace('B', '').replace(',', '').strip()) * 1_000_000_000
        elif 'M' in val:
            return float(val.replace('M+', '').replace('M', '').replace(',', '').strip()) * 1_000_000
        elif 'K' in val:
            return float(val.replace('K+', '').replace('K', '').replace(',', '').strip()) * 1_000
        else:
            return float(val.replace(',', '').replace('+', '').strip())
    return val

In [ ]:
# try to group data of game with duplicate title 

# convert 'Last Updated' to datetime format for sorting
df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')

# sort date by lated update
df = df.sort_values('Last Updated', ascending=False)

# For each Title keep only lastest row (for every duplicate decription)
df = df.drop_duplicates(subset=['Title','Description'], keep='first')

# parse total visits to number
df['Total Visits'] = df['Total Visits'].apply(parse_visits)

# sort title and last updated
df = df.sort_values(by=['Title', 'Last Updated'], ascending=[True, False])

print(df['Title'].value_counts())
print(df['Description'].value_counts())
print('----------- ** ----------')
print(f'Total data left {len(df)} row(s)')
print(f"Total unique Title: {df['Title'].nunique()}")
df.to_csv('clean.csv')